In [2]:
# Import packages
import pandas as pd
import nltk
import time
import json

In [3]:
reviews = pd.read_csv('../input/yelp_academic_dataset_review.csv')
# print(df.head(10))

In [5]:
business = pd.read_pickle('../input/business.pkl')

In [6]:
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [7]:
print(business['city'].unique())

['Santa Barbara' 'Affton' 'Tucson' ... 'Wales' 'UPPER MORELAND'
 'Apollo beach']


In [8]:
Tucson_data = business[business['city'] == 'Tucson']
print(Tucson_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9250 entries, 2 to 150331
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   9250 non-null   object 
 1   name          9250 non-null   object 
 2   address       9250 non-null   object 
 3   city          9250 non-null   object 
 4   state         9250 non-null   object 
 5   postal_code   9250 non-null   object 
 6   latitude      9250 non-null   float64
 7   longitude     9250 non-null   float64
 8   stars         9250 non-null   float64
 9   review_count  9250 non-null   int64  
 10  is_open       9250 non-null   int64  
 11  attributes    8328 non-null   object 
 12  categories    9244 non-null   object 
 13  hours         7916 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 1.1+ MB
None


In [10]:
Tucson_reviews = reviews[reviews['business_id'].isin(Tucson_data['business_id'])]
Tucson_reviews.shape

(404880, 9)

In [9]:
SantaBarbara_data = business[business['city'] == 'Santa Barbara']
print(SantaBarbara_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3829 entries, 0 to 150303
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   3829 non-null   object 
 1   name          3829 non-null   object 
 2   address       3829 non-null   object 
 3   city          3829 non-null   object 
 4   state         3829 non-null   object 
 5   postal_code   3829 non-null   object 
 6   latitude      3829 non-null   float64
 7   longitude     3829 non-null   float64
 8   stars         3829 non-null   float64
 9   review_count  3829 non-null   int64  
 10  is_open       3829 non-null   int64  
 11  attributes    3430 non-null   object 
 12  categories    3828 non-null   object 
 13  hours         3171 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 448.7+ KB
None


In [45]:
business = business.dropna(subset=['categories'])
restaurant_bar = business[business['categories'].str.contains('Restaurant')]

In [11]:
SantaBarbara_reviews = reviews[reviews['business_id'].isin(SantaBarbara_data['business_id'])]
SantaBarbara_reviews.shape

(269630, 9)

In [12]:
print(SantaBarbara_reviews.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269630 entries, 8 to 6990250
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   useful       269630 non-null  int64  
 1   stars        269630 non-null  float64
 2   cool         269630 non-null  int64  
 3   funny        269630 non-null  int64  
 4   review_id    269630 non-null  object 
 5   user_id      269630 non-null  object 
 6   text         269630 non-null  object 
 7   date         269630 non-null  object 
 8   business_id  269630 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 20.6+ MB
None


In [13]:
print(Tucson_reviews.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404880 entries, 2 to 6990242
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   useful       404880 non-null  int64  
 1   stars        404880 non-null  float64
 2   cool         404880 non-null  int64  
 3   funny        404880 non-null  int64  
 4   review_id    404880 non-null  object 
 5   user_id      404880 non-null  object 
 6   text         404880 non-null  object 
 7   date         404880 non-null  object 
 8   business_id  404880 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 30.9+ MB
None


In [14]:
from nltk.sentiment import SentimentIntensityAnalyzer
from branca.colormap import linear

## Santa Barbara

In [61]:
analyzer = SentimentIntensityAnalyzer()

In [65]:
# Calculate the sentiment score for each comment
SantaBarbara_reviews.loc[:, 'sentiment_score'] = SantaBarbara_reviews['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Merge merchant information and review information in order to categorize reviews to merchants
merged_data = pd.merge(SantaBarbara_reviews, SantaBarbara_data[['business_id', 'name', 'latitude', 'longitude', 'review_count']], on='business_id')

# Calculate the average sentiment score for each merchant
business_sentiment = merged_data.groupby(['business_id', 'name', 'latitude', 'longitude', 'review_count'])['sentiment_score'].mean().reset_index()

/var/folders/28/g_pv1x910n7d59ygr1drw3s40000gn/T/ipykernel_27438/1421629393.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SantaBarbara_reviews.loc[:, 'sentiment_score'] = SantaBarbara_reviews['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])


In [136]:
import numpy as np
import folium
from folium import plugins
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

sb_map2 = folium.Map(location=[34.41, -119.70], zoom_start=13)

q50 = business_sentiment['sentiment_score'].quantile(0.30)

for i, row in business_sentiment.iterrows():
    if row['sentiment_score'] < q50:
        fill_color = 'blue'
    else:
        fill_color = 'red'
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['review_count'] / 100,
        popup=row['name'],
        fill=True,
        fill_color=fill_color,
        color = None,
        fill_opacity=0.8,
        weight = 0
    ).add_to(sb_map2)

sb_map2

sb_map2.save('../result/santabarbara_sentiment_map.html')


## Tucson

In [15]:
analyzer = SentimentIntensityAnalyzer()

In [16]:
# Calculate the sentiment score for each comment
Tucson_reviews.loc[:, 'sentiment_score'] = Tucson_reviews['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Merge merchant information and review information in order to categorize reviews to merchants
merged_data = pd.merge(Tucson_reviews, Tucson_data[['business_id', 'name', 'latitude', 'longitude', 'review_count']], on='business_id')

# Calculate the average sentiment score for each merchant
business_sentiment = merged_data.groupby(['business_id', 'name', 'latitude', 'longitude', 'review_count'])['sentiment_score'].mean().reset_index()

/var/folders/28/g_pv1x910n7d59ygr1drw3s40000gn/T/ipykernel_1124/3005813254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tucson_reviews.loc[:, 'sentiment_score'] = Tucson_reviews['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])


In [37]:
import numpy as np
import folium
from folium import plugins
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

tucson_map = folium.Map(location=[32.2217, -110.9265], zoom_start=12)

q30 = business_sentiment['sentiment_score'].quantile(0.30)

for i, row in business_sentiment.iterrows():
    if row['sentiment_score'] < q30:
        fill_color = 'blue'
    else:
        fill_color = 'red'
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['review_count'] / 100,
        popup=row['name'],
        fill=True,
        fill_color=fill_color,
        color = None,
        fill_opacity=0.8,
        weight = 0
    ).add_to(tucson_map)

tucson_map

tucson_map.save('../result/tucson_sentiment_map.html')
